#####################################################

#### To compile Go modules with C types to work with Python run:
#### go build -o main.so -buildmode=c-shared main.go

#####################################################

In [ ]:
import os
import sys

import hail as hl
import pyspark


hail_jars = os.path.join(
    os.sep.join(sys.executable.split(os.sep)[:6]),
    "lib",
    f"python{sys.version_info.major}.{sys.version_info.minor}",
    "site-packages",
    "hail",
    "backend",
    "hail-all-spark.jar",
)

CONF = pyspark.SparkConf().setAll(
    [
        ("spark.master", f"local[{20}]"),
        ("spark.app.name", "Hail_demonstration"),
        ("spark.jars", hail_jars),
        ("spark.driver.extraClassPath", hail_jars),
        ("spark.driver.memory", "30g"),
        ("spark.executor.extraClassPath", "./hail-all-spark.jar"),
        ("spark.serializer", "org.apache.spark.serializer.KryoSerializer"),
        ("spark.kryo.registrator", "is.hail.kryo.HailKryoRegistrator"),
    ]
)

sc = pyspark.SparkContext(conf=CONF)

hl.init(default_reference="GRCh38", sc=sc)

In [ ]:
from matrix_table_consumer import MatrixTableConsumer, NUM_CPU

In [ ]:
# Downloaded from https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/

vcf_big_path = "./data/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz"  # GRCh37
vcf_mini_path = "./data/test1.vcf"  # GRCh38

matrix_table_path = "./data/matrix_table.mt"
json_path = "./data/matrix.json"

In [ ]:
mt = hl.import_vcf(
    vcf_mini_path,
    reference_genome="GRCh38",
    array_elements_required=False,
    force=True,
)
mt.show()

,,
,,'tumor'
locus,alleles,GT
locus<GRCh38>,array<str>,call
chr1:13100,"[""C"",""T""]",0/1
chr1:25734793,"[""C"",""T""]",0/1
chr1:37323930,"[""C"",""T""]",0/1
chr2:1234,"[""C"",""T""]",0/1
chr2:1235,"[""C"",""T""]",0/1
chr3:1234,"[""C"",""T""]",0/1
chr4:1234,"[""C"",""T""]",0/1


In [ ]:
consumer = MatrixTableConsumer(vcf_path=vcf_mini_path, reference_genome="GRCh38")

# Save MatrixTable

In [ ]:
content = {}
content = consumer.prepare_metadata_for_saving(json_path=json_path, mt=mt)

Extracting fields:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting fields: 100%|██████████| 2/2 [00:00<00:00, 34.10it/s]


# Download MatrixTable

In [ ]:
mt_new = consumer.prepare_metadata_for_loading(json_path=json_path)

mt_new.write(matrix_table_path, overwrite=True)
mt_new.count()

Prepare metadata for loading: 100%|██████████| 3/3 [00:00<00:00, 325.63it/s]


(13, 1)

# Collect rows

In [ ]:
consumer = MatrixTableConsumer(vcf_path=vcf_big_path, reference_genome="GRCh37")

In [ ]:
rows = consumer.collect(num_rows=200_000, num_cpu=1)
rows[:10]

[{'CHROM': '1',
  'ID': '.',
  'REF': 'A',
  'ALT': 'T',
  'FILTER': 'PASS',
  'INFO': 'AC=7;AF=0.00139776;AN=5008;NS=2504;DP=16853;EAS_AF=0.0069;AMR_AF=0;AFR_AF=0;EUR_AF=0;SAS_AF=0;AA=A|||;VT=SNP',
  'POS': 6122681,
  'QUAL': 100},
 {'CHROM': '1',
  'ID': '.',
  'REF': 'G',
  'ALT': 'T',
  'FILTER': 'PASS',
  'INFO': 'AC=3;AF=0.000599042;AN=5008;NS=2504;DP=16928;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0;EUR_AF=0.002;SAS_AF=0;AA=G|||;VT=SNP',
  'POS': 6122705,
  'QUAL': 100},
 {'CHROM': '1',
  'ID': '.',
  'REF': 'C',
  'ALT': 'T',
  'FILTER': 'PASS',
  'INFO': 'AC=1;AF=0.000199681;AN=5008;NS=2504;DP=17669;EAS_AF=0;AMR_AF=0;AFR_AF=0.0008;EUR_AF=0;SAS_AF=0;AA=C|||;VT=SNP',
  'POS': 6122757,
  'QUAL': 100},
 {'CHROM': '1',
  'ID': '.',
  'REF': 'T',
  'ALT': 'C',
  'FILTER': 'PASS',
  'INFO': 'AC=1;AF=0.000199681;AN=5008;NS=2504;DP=19036;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0;SAS_AF=0.001;AA=T|||;VT=SNP',
  'POS': 6122810,
  'QUAL': 100},
 {'CHROM': '1',
  'ID': '.',
  'REF': 'T',
  'ALT': 'C',
  'FI

In [ ]:
rows_count = consumer.count()
rows_count

In [ ]:
rows = consumer.collect_all(num_cpu=NUM_CPU)
rows[:2]

In [ ]:
print(len(rows))

In [ ]:
rows_hail = consumer.convert_rows_to_hail(rows=rows)
rows_hail[:5]

Converting rows to hail: 100%|██████████| 200000/200000 [00:01<00:00, 108585.18it/s]


[Struct(locus=Locus(contig=1, position=6122681, reference_genome=GRCh37), alleles=['A', 'T'], rsid='.', qual=100, filters='PASS', info=Struct(info='AC=7;AF=0.00139776;AN=5008;NS=2504;DP=16853;EAS_AF=0.0069;AMR_AF=0;AFR_AF=0;EUR_AF=0;SAS_AF=0;AA=A|||;VT=SNP'), entries=[]),
 Struct(locus=Locus(contig=1, position=6122705, reference_genome=GRCh37), alleles=['G', 'T'], rsid='.', qual=100, filters='PASS', info=Struct(info='AC=3;AF=0.000599042;AN=5008;NS=2504;DP=16928;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0;EUR_AF=0.002;SAS_AF=0;AA=G|||;VT=SNP'), entries=[]),
 Struct(locus=Locus(contig=1, position=6122757, reference_genome=GRCh37), alleles=['C', 'T'], rsid='.', qual=100, filters='PASS', info=Struct(info='AC=1;AF=0.000199681;AN=5008;NS=2504;DP=17669;EAS_AF=0;AMR_AF=0;AFR_AF=0.0008;EUR_AF=0;SAS_AF=0;AA=C|||;VT=SNP'), entries=[]),
 Struct(locus=Locus(contig=1, position=6122810, reference_genome=GRCh37), alleles=['T', 'C'], rsid='.', qual=100, filters='PASS', info=Struct(info='AC=1;AF=0.000199681;AN=5008

In [ ]:
t = consumer.create_hail_table(rows=rows_hail)
t.show()

,,,,,,
,,,,,info,
locus,alleles,rsid,qual,filters,info,entries
locus<GRCh37>,array<str>,str,int32,str,str,array<struct{}>
1:6122681,"[""A"",""T""]",""".""",100,"""PASS""","""AC=7;AF=0.00139776;AN=5008;NS=2504;DP=16853;EAS_AF=0.0069;AMR_AF=0;AFR_AF=0;EUR_AF=0;SAS_AF=0;AA=A|||;VT=SNP""",[]
1:6122705,"[""G"",""T""]",""".""",100,"""PASS""","""AC=3;AF=0.000599042;AN=5008;NS=2504;DP=16928;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0;EUR_AF=0.002;SAS_AF=0;AA=G|||;VT=SNP""",[]
1:6122757,"[""C"",""T""]",""".""",100,"""PASS""","""AC=1;AF=0.000199681;AN=5008;NS=2504;DP=17669;EAS_AF=0;AMR_AF=0;AFR_AF=0.0008;EUR_AF=0;SAS_AF=0;AA=C|||;VT=SNP""",[]
1:6122810,"[""T"",""C""]",""".""",100,"""PASS""","""AC=1;AF=0.000199681;AN=5008;NS=2504;DP=19036;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0;SAS_AF=0.001;AA=T|||;VT=SNP""",[]
1:6122845,"[""T"",""C""]",""".""",100,"""PASS""","""AC=10;AF=0.00199681;AN=5008;NS=2504;DP=19063;EAS_AF=0;AMR_AF=0;AFR_AF=0.0076;EUR_AF=0;SAS_AF=0;AA=T|||;VT=SNP""",[]
1:6122914,"[""A"",""G""]",""".""",100,"""PASS""","""AC=1;AF=0.000199681;AN=5008;NS=2504;DP=18974;EAS_AF=0;AMR_AF=0;AFR_AF=0.0008;EUR_AF=0;SAS_AF=0;AA=A|||;VT=SNP""",[]
1:6123079,"[""C"",""T""]",""".""",100,"""PASS""","""AC=5;AF=0.000998403;AN=5008;NS=2504;DP=14296;EAS_AF=0.001;AMR_AF=0.0014;AFR_AF=0.0015;EUR_AF=0;SAS_AF=0.001;AA=C|||;VT=SNP""",[]


# Create MatrixTable from Table

In [ ]:
mt_from_table = consumer.combine_hail_matrix_table_and_table(mt=mt, table=t)
mt_from_table.show()

In [ ]:
mt_from_table.count()

# Zarr format

In [1]:
from matrix_table_consumer import MatrixTableConsumer, NUM_CPU

vcf_big_path = "/home/phil/GitHub/matrix_table_consumer/data/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz"
vcz_path = "/home/phil/GitHub/matrix_table_consumer/data/test.vcz"

consumer = MatrixTableConsumer(vcf_path=vcf_big_path)

Loading BokehJS ...

In [ ]:
import numpy as np


data = consumer.load_zarr_data(vcz_path=vcz_path)

genotypes = data["call_genotype"]
n_variants, n_samples, _ = genotypes.shape

phenotypes = np.random.normal(0, 1, n_samples)
covariates = np.random.normal(0, 1, (n_samples, 2))  # 2 ковариаты

In [ ]:
gwas_df = consumer.run_gwas(data, phenotypes, covariates)

Processing chunks:   0%|          | 0/5000 [00:00<?, ?it/s]

Processing chunks: 100%|██████████| 5000/5000 [01:18<00:00, 63.38it/s]

Of 4988 significant variants found (p < 0.05): 305


In [ ]:
gwas_df

,variant_index,variant_id,beta,se,p_value,t_stat,maf
0,0,.,-0.028931,0.032327,0.370891,-0.894967,0.425319
1,1,.,-0.392726,0.409082,0.337139,-0.960017,0.001198
2,2,.,-0.077935,0.047707,0.102462,-1.633631,0.437500
3,3,.,0.769047,1.001260,0.442513,0.768079,0.000200
4,4,.,0.769047,1.001260,0.442513,0.768079,0.000200
...,...,...,...,...,...,...,...
4983,4995,.,-0.019695,0.055557,0.722994,-0.354500,0.068091
4984,4996,.,-0.053789,0.302762,0.859003,-0.177662,0.002196
4985,4997,.,-2.272777,0.999828,0.023101,-2.273168,0.000200
4986,4998,.,0.056547,0.035874,0.115085,1.576293,0.189497
